# Plot counts per lineage through time

## Imports

In [ ]:
import altair as alt
import pandas as pd

## Configuration

In [ ]:
min_date = "2018-01-01"

## Load metadata

In [ ]:
def load_lineage_dates(metadata_file, lineage):
    df = pd.read_csv(metadata_file, sep="\t")
    df["lineage"] = lineage
    dates = df.loc[~df["date"].str.contains("X"), ["lineage", "date"]]
    
    return dates

In [ ]:
h3n2_dates = load_lineage_dates("2022-05-31/metadata/metadata_h3n2_ha.tsv", "H3N2")

In [ ]:
h1n1pdm_dates = load_lineage_dates("2022-05-31/metadata/metadata_h1n1pdm_ha.tsv", "H1N1pdm")

In [ ]:
vic_dates = load_lineage_dates("2022-05-31/metadata/metadata_vic_ha.tsv", "Vic")

In [ ]:
yam_dates = load_lineage_dates("2022-05-31/metadata/metadata_yam_ha.tsv", "Yam")

In [ ]:
dates = pd.concat([h3n2_dates, h1n1pdm_dates, vic_dates, yam_dates])

In [ ]:
dates.head()

In [ ]:
dates.shape

In [ ]:
recent_dates = dates[dates["date"] > min_date].copy()

In [ ]:
recent_dates["date"] = pd.to_datetime(recent_dates["date"])

In [ ]:
recent_dates.shape

In [ ]:
binned_counts = recent_dates.set_index("date").groupby("lineage").resample("1MS").count().rename(columns={"lineage": "samples"}).reset_index()

In [ ]:
binned_counts

In [ ]:
alt.Chart(binned_counts).mark_line(point=True).encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("samples:Q", title="Number of samples"),
    color=alt.Color("lineage:N", sort=["H3N2", "H1N1pdm", "Vic", "Yam"], title="Lineage"),
    tooltip=["lineage:N", "date:T", "samples:Q"],
).configure_axis(
    labelFontSize=14,
    titleFontSize=14,
).configure_legend(
    labelFontSize=14,
    titleFontSize=14,
).properties(
    width=900,
    height=400,
)